# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,787.797719,0.104268,1234,787.901987,787.693451,1575.803975,1575.386902
6,462.839347,-0.574010,1234,462.265337,463.413357,924.530674,926.826714
7,976.905053,0.271311,1234,977.176364,976.633742,1954.352727,1953.267485
8,531.231843,-0.531234,1234,530.700609,531.763077,1061.401218,1063.526155
9,291.021242,-0.842189,1234,290.179053,291.863431,580.358106,583.726861
10,980.735500,0.925119,1234,981.660619,979.810382,1963.321237,1959.620763
11,479.708014,0.443989,1234,480.152002,479.264025,960.304005,958.528050
12,598.587779,-0.266317,1234,598.321461,598.854096,1196.642923,1197.708192
13,431.421134,-0.737696,1234,430.683438,432.158830,861.366876,864.317659
14,974.512503,-0.599957,1234,973.912546,975.112459,1947.825091,1950.224919


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-787.901987,787.693451
6,-462.265337,463.413357
7,-977.176364,976.633742
8,-530.700609,531.763077
9,-290.179053,291.863431
10,-981.660619,979.810382
11,-480.152002,479.264025
12,-598.321461,598.854096
13,-430.683438,432.158830
14,-973.912546,975.112459


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-787.901987,787.693451
6,-462.265337,463.413357
7,-977.176364,976.633742
8,-530.700609,531.763077
9,-290.179053,291.863431
10,-981.660619,979.810382
11,-480.152002,479.264025
12,-598.321461,598.854096
13,-430.683438,432.158830
14,-973.912546,975.112459


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-787.90198747,  787.69345091],
       [-462.26533702,  463.41335708],
       [-977.17636373,  976.63374232],
       [-530.70060892,  531.76307726],
       [-290.1790531 ,  291.86343062],
       [-981.66061871,  979.81038166],
       [-480.15200246,  479.26402491],
       [-598.32146126,  598.85409583],
       [-430.68343815,  432.15882963],
       [-973.91254575,  975.11245932]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1573.803975,-1575.386902
6,922.530674,-926.826714
7,1952.352727,-1953.267485
8,1059.401218,-1063.526155
9,578.358106,-583.726861
10,1961.321237,-1959.620763
11,958.304005,-958.528050
12,1194.642923,-1197.708192
13,859.366876,-864.317659
14,1945.825091,-1950.224919


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.787798,0.552134
6,0.462839,0.212995
7,0.976905,0.635655
8,0.531232,0.234383
9,0.291021,0.078906
10,0.980736,0.962559
11,0.479708,0.721994
12,0.598588,0.366841
13,0.431421,0.131152
14,0.974513,0.200022


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,787.797719,0.104268
6,462.839347,-0.574010
7,976.905053,0.271311
8,531.231843,-0.531234
9,291.021242,-0.842189
10,980.735500,0.925119
11,479.708014,0.443989
12,598.587779,-0.266317
13,431.421134,-0.737696
14,974.512503,-0.599957


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
